### Reference:
https://www.math.ubc.ca/~pwalls/math-python/linear-algebra/linear-algebra-scipy/

In [49]:
import numpy as np
import scipy.linalg as la
import math

### Matrix Operations

In [15]:
M = np.array([[3,4],[-1,5]])
N = np.array([[5,7],[-9,6], [2, 3]])
L = np.array([[1], [2]])
print(M)
print("------")
print(N)
print("------")
print(L)

[[ 3  4]
 [-1  5]]
------
[[ 5  7]
 [-9  6]
 [ 2  3]]
------
[[1]
 [2]]


##### Normal Multiplication

In [5]:
# multiplies corresponding elements
M * M

array([[ 9, 16],
       [ 1, 25]])

##### Matrix Multiplication

In [13]:
M@M

array([[ 5, 32],
       [-8, 21]])

In [14]:
M@L

array([[11],
       [ 9]])

In [16]:
# Compute 2L - 3ML
2*L - 3*(M@L)

array([[-31],
       [-23]])

##### Matrix Powers

Only defined for square matrices

In [17]:
from numpy.linalg import matrix_power as mpow

In [18]:
mpow(M,2)

array([[ 5, 32],
       [-8, 21]])

In [22]:
# mpow(M,2) is same as M@M
M@M

array([[ 5, 32],
       [-8, 21]])

##### Matrix Transpose

In [23]:
M.T

array([[ 3, -1],
       [ 4,  5]])

In [28]:
# M @ M.T is a symmetric Matrix
M @ M.T

array([[25, 17],
       [17, 26]])

##### Matrix Inverse

In [25]:
la.inv(M)

array([[ 0.26315789, -0.21052632],
       [ 0.05263158,  0.15789474]])

In [27]:
# M @ inv(M) is an identity matrix
M @ la.inv(M)

array([[1.00000000e+00, 0.00000000e+00],
       [5.55111512e-17, 1.00000000e+00]])

##### Trace

In [29]:
M

array([[ 3,  4],
       [-1,  5]])

In [31]:
# Trace is the sum of diagonals of the matrix
np.trace(M)

8

##### Determinant

In [32]:
la.det(M)

19.0

In [33]:
M

array([[ 3,  4],
       [-1,  5]])

##### Verifying the Cayley-Hamilton Theorem

In [34]:
trace_M = np.trace(M)
det_M = la.det(M)
I = np.eye(2)
M @ M - trace_M * M + det_M * I

array([[0., 0.],
       [0., 0.]])

### Row Operations as Matrix Products

In [9]:
A = np.array([[1, 2], [3, 4]])

In [10]:
# here the scale factor, m=5
Er = [[1, 5], [0, 1]]

In [11]:
# first let's compute the operation: R : R <- R1 + 5R2

In [12]:
A1 = A.copy()

In [13]:
A1[:1] = A1[:1] + 5*A1[1:2]

In [14]:
A1

array([[16, 22],
       [ 3,  4]])

In [16]:
# Next, let's multiply by the matrix Er and check if both are same

In [17]:
Er @ A

array([[16, 22],
       [ 3,  4]])

Both are same!

## Determinants

In [26]:
a = np.array([[1, 2], [3, 4]])
np.linalg.det(a)

-2.0000000000000004

In [27]:
b = np.array([[3, 4], [1, 2]])
np.linalg.det(b)

2.0000000000000004

In [28]:
P = [[1, 2, 3], [2, 2, 4], [2, 2, 5]]

In [30]:
volume = la.det(P)

In [31]:
volume 

-1.9999999999999996

### Computing determinants manually

In [44]:
def get_minor(m, i, j):
    '''
    get minor associated with a particular element given by row_idx, column_idx
    reference: https://stackoverflow.com/questions/53934405/find-minor-matrix-in-python#:~:text=You%20have%20to%20extract%20a,list%20of%20the%20remaining%20rows.
    '''
    return np.array([row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])])

In [45]:
def find_determinant(matrix, row_idx):
    '''
    computes the determinant of a matrix along a particular row
    '''
    # verify if the matrix is a square matrix or not
    rows, columns = matrix.shape
    if rows != columns:
        raise TypeError("Determinant can be only computed for a square matrix")
    if rows == 2:
        # if it's a square matrix, we directly compute the determinant using the formula ad - bc
        a, b, c, d = matrix[0][0], matrix[0][1], matrix[1][0], matrix[1][1]
        return a*d - b*c
    determinant = 0
    for column_idx in range(columns):
        sign = math.pow(-1, row_idx + column_idx)
        minor = get_minor(matrix.tolist(), row_idx, column_idx) 
        det_minor = find_determinant(minor, 0)
        element = matrix[row_idx][column_idx]
        determinant += sign*element*det_minor
    return determinant

#### Check if the function is correct

In [50]:
A = np.array([[1, 2, 13], 
              [14, 52, 6], 
              [71, 8, 9]])

In [51]:
la.det(A)

-45520.0

In [52]:
find_determinant(A, 0)

-45520.0

Looks like the function is correct!

In [53]:
find_determinant(A, 0)

-45520.0

In [54]:
find_determinant(A, 1)

-45520.0

In [55]:
find_determinant(A, 2)

-45520.0

Perfect!

#### Cross Product as a determinant

##### Let's verify it

In [56]:
A = np.array([[1, 2, 13], 
              [14, 52, 6], 
              [71, 8, 9]])

In [57]:
la.det(A)

-45520.0

In [61]:
u = A[:1]
v = A[1:2]
w = A[2:3]

In [73]:
# computing u.(v x w)
np.dot(np.cross(v, w), u.T)[0][0]

-45520

Perfect!

## Matrix Inverse

#### Using Adjuage Matrix Approach

In [117]:
def find_inverse(matrix):
    rows, columns = matrix.shape
    if rows != columns:
        raise TypeError("Inverse is only defined for square matrix")
    detA = find_determinant(A, 0)
    n = rows
    inverse_matrix = np.zeros(shape=(rows, columns))
    for i in range(n):
        for j in range(n):
            sign = math.pow(-1, i+j)
            minor = get_minor(matrix.tolist(), i, j)
            det_minor = find_determinant(minor, 0)
            inverse_matrix[i][j] = sign*det_minor
    return inverse_matrix.T * (1/detA)

#### Let's verify the inverse

In [118]:
A = np.array([[1, 2, 13], 
              [14, 52, 6], 
              [71, 8, 9]])

In [119]:
Ainv = find_inverse(A)

In [121]:
np.round(Ainv@A)

array([[ 1.,  0., -0.],
       [ 0.,  1., -0.],
       [-0., -0.,  1.]])

Perfect!

#### Using Row Operations

##### Example

In [126]:
A = np.array([[1, 2], [3, 9]])

In [131]:
I = np.eye(A.ndim)

In [136]:
augmented_matrix = np.concatenate([A, I], axis=1)

In [137]:
augmented_matrix

array([[1., 2., 1., 0.],
       [3., 9., 0., 1.]])

In [139]:
augmented_matrix[1:2] = augmented_matrix[1:2] - 3*augmented_matrix[:1]

In [140]:
augmented_matrix

array([[ 1.,  2.,  1.,  0.],
       [ 0.,  3., -3.,  1.]])

In [141]:
augmented_matrix[1:2] = augmented_matrix[1:2] * (1/3)

In [142]:
augmented_matrix

array([[ 1.        ,  2.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        , -1.        ,  0.33333333]])

In [143]:
augmented_matrix[:1] = augmented_matrix[:1] - 2 * augmented_matrix[1:2]

In [144]:
augmented_matrix

array([[ 1.        ,  0.        ,  3.        , -0.66666667],
       [ 0.        ,  1.        , -1.        ,  0.33333333]])

In [145]:
Ainv = augmented_matrix[:, 2:]

In [147]:
A@Ainv

array([[1., 0.],
       [0., 1.]])

Perfect!